In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from scipy import sparse
from sklearn.feature_extraction import FeatureHasher
from scipy.sparse import coo_matrix,csr_matrix,csc_matrix, hstack
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn import linear_model
import gc
from sklearn import preprocessing

In [2]:
%ls

1_xgboost.ipynb*         origin/
3_xgb_prediction.ipynb*  pivot_test.pickle*
4_keras_nn.ipynb*        pivot_train_with_nan.pickle*
5_random_forest.ipynb*   train_pivot_45678_to_9.csv*
matplotlib-siyuan/       train_pivot_56789_to_10_new.pickle*
model_stack/             train_pivot_6789_to_11_new.pickle*
old_ipython/             train_pivot_xgb_time1.csv*
old_submission/          train_pivot_xgb_time2.csv*


### begin training, for week 11
----------------

In [18]:
predictors_target_11 = ['LR_prod', 'LR_prod_corr',
       'NombreCliente',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'target', 'weight', 'weight_per_piece']

In [19]:
predictors_11 = ['LR_prod', 'LR_prod_corr',
       'NombreCliente',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'weight', 'weight_per_piece']

In [10]:
f = lambda x : (x-x.mean())/x.std(ddof=0)

In [14]:
train_pivot_xgb_time2 = pd.read_csv('train_pivot_xgb_time2.csv',index_col = 0)

In [7]:
train_pivot_6789_to_11 = pd.read_pickle('train_pivot_6789_to_11_new.pickle')

In [8]:
train_pivot_xgb_time2.head()

,Agencia_ID,Canal_ID,Cliente_ID,LR_prod,LR_prod_corr,NombreCliente,Producto_ID,Ruta_SAK,agen_cliente_for_log_de,agen_for_log_de,...,t_m_3_cum,t_m_4_cum,t_m_5_cum,t_min_2,t_min_3,t_min_4,t_min_5,target,weight,weight_per_piece
0,2061,2,26,2.001190,7.293554,18434,1182,7212,2.852285,3.491654,...,NaN,NaN,3.688879,NaN,NaN,NaN,3.688879,0.000000,210.0,210.00
1,2061,2,26,1.839411,6.703932,18434,4767,7212,2.852285,3.491654,...,NaN,NaN,3.761200,NaN,NaN,NaN,3.761200,3.761200,250.0,NaN
2,2061,2,26,1.911283,6.965878,18434,31393,7212,2.852285,3.491654,...,8.650325,5.877736,3.044522,2.772589,2.772589,2.833213,3.044522,3.135494,640.0,NaN
3,2061,2,26,3.113374,11.347029,18434,34204,7212,2.852285,3.491654,...,11.024839,7.218177,3.784190,3.555348,3.806662,3.433987,3.784190,3.828641,450.0,56.25
4,2061,2,26,2.031231,7.403043,18434,34206,7212,2.852285,3.491654,...,12.963710,9.202510,4.795791,4.248495,3.761200,4.406719,4.795791,4.499810,340.0,42.50


In [15]:
train_pivot_xgb_time2.columns.values

array(['Agencia_ID', 'Canal_ID', 'Cliente_ID', 'LR_prod', 'LR_prod_corr',
       'NombreCliente', 'Producto_ID', 'Ruta_SAK',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'target', 'weight', 'weight_per_piece'], dtype=object)

In [4]:
def normalize_dataset(train_dataset,test_dataset):
    train_dataset_normalize = train_dataset[predictors_11].copy()
    train_dataset_normalize['label'] = 0    
    
    test_dataset_normalize = test_dataset[predictors_11].copy()
    test_dataset_normalize['label'] = 1
    
    whole_dataset = pd.concat([train_dataset_normalize,test_dataset_normalize])
    whole_dataset_normalize = whole_dataset.apply(f,axis = 0)
    
    train_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label'] == 0]
    test_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label']==1]
    
    train_dataset_normalize.drop(['label'],axis = 1,inplace = True)
    test_dataset_normalize.drop(['label'],axis =1,inplace = True)
    
    train_dataset_normalize['target'] = train_dataset['target'].copy() 
    
#     target = train_dataset['target']
    return train_dataset_normalize,test_dataset_normalize

In [21]:
train_dataset_normalize, test_dataset_normalize = normalize_dataset(train_pivot_xgb_time2,train_pivot_6789_to_11)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
train_dataset_normalize.head()

Semana,LR_prod,LR_prod_corr,NombreCliente,agen_cliente_for_log_de,agen_for_log_de,agen_producto_for_log_de,agen_ruta_for_log_de,cliente_for_log_de,cliente_for_log_sum,cliente_producto_for_log_de,...,t_m_3_cum,t_m_4_cum,t_m_5_cum,t_min_2,t_min_3,t_min_4,t_min_5,weight,weight_per_piece,target
0,0.440004,0.007984,-1.198863,2.893915,7.040262,4.922515,3.707101,2.951726,0.023732,2.468511,...,NaN,NaN,0.841755,NaN,NaN,NaN,2.148223,0.041948,0.552880,0.000000
1,0.136858,0.004843,-1.198863,2.893915,7.040262,4.561988,3.707101,2.951726,0.023732,2.558317,...,NaN,NaN,0.888582,NaN,NaN,NaN,2.230611,0.180385,NaN,3.761200
2,0.271533,0.006239,-1.198863,2.893915,7.040262,3.000979,3.707101,2.951726,0.023732,1.433925,...,1.028141,0.851082,0.424536,1.090852,1.086921,1.149949,1.414172,1.530144,NaN,3.135494
3,2.524041,0.029578,-1.198863,2.893915,7.040262,3.917930,3.707101,2.951726,0.023732,2.414081,...,1.752516,1.424506,0.903467,1.968861,2.245187,1.825675,2.256800,0.872569,-0.290791,3.828641
4,0.496296,0.008568,-1.198863,2.893915,7.040262,5.719378,3.707101,2.951726,0.023732,3.231175,...,2.343992,2.273380,1.558475,2.746354,2.194265,2.919764,3.409216,0.491868,-0.366242,4.499810


In [26]:
train_pivot_xgb_time2_sample = train_dataset_normalize.sample(2000000)

train_feature_11 = train_pivot_xgb_time2_sample.drop(['target'],axis = 1)
train_label_11 = train_pivot_xgb_time2_sample[['target']]

dtrain_11 = xgb.DMatrix(train_feature_11,label = train_label_11,missing=np.nan)

In [27]:
num_round = 1000

cvresult = xgb.cv(param_11, dtrain_11, num_round, nfold=5,verbose_eval = 1,show_stdv=False,
                        seed = 0, early_stopping_rounds=5)
print(cvresult.tail())

[0]	train-rmse:1.14019	test-rmse:1.14021
[1]	train-rmse:0.961458	test-rmse:0.961473
[2]	train-rmse:0.824843	test-rmse:0.824873
[3]	train-rmse:0.722328	test-rmse:0.722397
[4]	train-rmse:0.647489	test-rmse:0.647603
[5]	train-rmse:0.592862	test-rmse:0.593041
[6]	train-rmse:0.555021	test-rmse:0.555254
[7]	train-rmse:0.528662	test-rmse:0.528924
[8]	train-rmse:0.510662	test-rmse:0.510977
[9]	train-rmse:0.498551	test-rmse:0.498903
[10]	train-rmse:0.49003	test-rmse:0.490423
[11]	train-rmse:0.484163	test-rmse:0.484588
[12]	train-rmse:0.480003	test-rmse:0.480457
[13]	train-rmse:0.477039	test-rmse:0.47752
[14]	train-rmse:0.474897	test-rmse:0.475405
[15]	train-rmse:0.473305	test-rmse:0.473844
[16]	train-rmse:0.472061	test-rmse:0.472641
[17]	train-rmse:0.471142	test-rmse:0.471752
[18]	train-rmse:0.470369	test-rmse:0.471
[19]	train-rmse:0.469729	test-rmse:0.470375
[20]	train-rmse:0.469123	test-rmse:0.469783
[21]	train-rmse:0.468683	test-rmse:0.469369
[22]	train-rmse:0.468239	test-rmse:0.468944
[23]	

In [24]:
param_11 = {'booster':'gbtree',
         'nthread': 10,
         'max_depth':5, 
         'eta':0.2,
         'silent':1,
         'subsample':0.7, 
         'objective':'reg:linear',
         'eval_metric':'rmse',
         'colsample_bytree':0.7}

In [28]:
num_round = 566

dtest_11 = xgb.DMatrix(test_dataset_normalize[predictors_11], missing=np.nan)
submission_11 = train_pivot_6789_to_11[['id']].copy()
j =0 
for j in range(20):
    
    train_pivot_xgb_time2_sample = train_dataset_normalize[predictors_target_11].sample(2000000)
    train_feature_11 = train_pivot_xgb_time2_sample.drop(['target'],axis = 1)
    train_label_11 = train_pivot_xgb_time2_sample[['target']]

    dtrain_11 = xgb.DMatrix(train_feature_11,label = train_label_11,missing= np.nan)
    
    bst_11 = xgb.train(param_11, dtrain_11, num_round)
    print str(j) + 'training finished!'
    submission_11['predict_' + str(j)] = bst_11.predict(dtest_11)

print 'finished'

0training finished!
1training finished!
2training finished!
3training finished!
4training finished!
5training finished!
6training finished!
7training finished!
8training finished!
9training finished!
10training finished!
11training finished!
12training finished!
13training finished!
14training finished!
15training finished!
16training finished!
17training finished!
18training finished!
19training finished!
finished


In [12]:
# make prediction
dtest_11 = xgb.DMatrix(train_pivot_6789_to_11[predictors], missing=NaN)
submission_11 = train_pivot_6789_to_11[['id']].copy()
submission_11['predict'] = bst.predict(dtest)
xgb.plot_importance(bst)

In [29]:
submission_11.to_csv('submission_11_new.csv')

In [25]:
submission_11 = pd.read_csv('submission_11_new.csv',index_col =0)

### for week 10
-------

In [4]:
%ls

1_xgboost.ipynb*        pivot_train_with_nan.pickle*
3_prediction.ipynb*     submission_nn.csv*
4_keras_nn.ipynb*       train_pivot_45678_to_9.csv*
5_random_forest.ipynb*  train_pivot_56789_to_10_new.pickle*
old_backup/             train_pivot_6789_to_11_new.pickle*
old_ipython/            train_pivot_xgb_time1.csv*
origin/                 train_pivot_xgb_time2.csv*
pivot_test.pickle*


In [2]:
predictors_target_10 = ['agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum', 'corr',
       't_min_1', 't_min_2', 't_min_3', 't_min_4', 't_min_5', 't1_min_t2',
       't1_min_t3', 't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4',
       't2_min_t5', 't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod',
       'LR_prod_corr', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum',
       't_m_2_cum', 't_m_1_cum', 'NombreCliente', 'weight',
       'weight_per_piece', 'pieces','target']

In [3]:
predictors_10 = ['agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum', 'corr',
       't_min_1', 't_min_2', 't_min_3', 't_min_4', 't_min_5', 't1_min_t2',
       't1_min_t3', 't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4',
       't2_min_t5', 't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod',
       'LR_prod_corr', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum',
       't_m_2_cum', 't_m_1_cum', 'NombreCliente', 'weight',
       'weight_per_piece', 'pieces']

In [13]:
f = lambda x : (x-x.mean())/x.std(ddof=0)

In [14]:
def normalize_dataset_10(train_dataset,test_dataset):
    train_dataset_normalize = train_dataset[predictors_10]
    train_dataset_normalize['label'] = 0    
    
    test_dataset_normalize = test_dataset[predictors_10]
    test_dataset_normalize['label'] = 1
    
    whole_dataset = pd.concat([train_dataset_normalize,test_dataset_normalize],copy = False)
    whole_dataset_normalize = whole_dataset.apply(f,axis = 0)
    
    train_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label'] == 0]
    test_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label']==1]
    
    train_dataset_normalize.drop(['label'],axis = 1,inplace = True)
    test_dataset_normalize.drop(['label'],axis =1,inplace = True)
    
    train_dataset_normalize['target'] = train_dataset['target']
    
#     target = train_dataset['target']
    return train_dataset_normalize,test_dataset_normalize

In [5]:
train_pivot_xgb_time1 = pd.read_csv('train_pivot_xgb_time1.csv',
                                    usecols = predictors_target_10,dtype = dtypes)
train_pivot_xgb_time1.reset_index(drop = True,inplace = True)

In [5]:
train_pivot_56789_to_10 = pd.read_pickle('train_pivot_56789_to_10_new.pickle')
train_pivot_56789_to_10.reset_index(drop = True,inplace = True)

In [7]:
train_pivot_56789_to_10.columns.values

array(['Cliente_ID', 'Producto_ID', 'id', 'Semana', 'Agencia_ID',
       'Canal_ID', 'Ruta_SAK', 'agen_for_log_de', 'ruta_for_log_de',
       'cliente_for_log_de', 'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum', 'corr',
       't_min_1', 't_min_2', 't_min_3', 't_min_4', 't_min_5', 't1_min_t2',
       't1_min_t3', 't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4',
       't2_min_t5', 't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod',
       'LR_prod_corr', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       't_m_1_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces'], dtype=object)

In [6]:
train_pivot_xgb_time1.columns.values

array(['agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum', 'corr',
       't_min_1', 't_min_2', 't_min_3', 't_min_4', 't_min_5', 't1_min_t2',
       't1_min_t3', 't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4',
       't2_min_t5', 't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod',
       'LR_prod_corr', 'target', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum',
       't_m_2_cum', 't_m_1_cum', 'NombreCliente', 'weight',
       'weight_per_piece', 'pieces'], dtype=object)

In [17]:
# train_dataset_10_normalize, test_dataset_10_normalize = normalize_dataset_10(train_pivot_xgb_time1,
#                                                                           train_pivot_56789_to_10)

In [ ]:
train_dataset_10_normalize.head()

In [8]:
param_10 = {'booster':'gbtree',
         'nthread': 7,
         'max_depth':5, 
         'eta':0.2,
         'silent':1,
         'subsample':0.7, 
         'objective':'reg:linear',
         'eval_metric':'rmse',
         'colsample_bytree':0.7}

In [6]:
# train_pivot_xgb_time1_sample = train_pivot_xgb_time1[predictors_target_10].sample(1000000)

# train_feature_10 = train_pivot_xgb_time1_sample.drop(['target'],axis = 1)
# train_label_10 = train_pivot_xgb_time1_sample[['target']]

# dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)

In [7]:
train_label_10 = train_pivot_xgb_time1['target']
train_feature_10 = train_pivot_xgb_time1.drop(['target'],axis = 1)



dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)


In [6]:
# num_round = 1500

# cvresult = xgb.cv(param_10, dtrain_10, num_round, nfold=5,show_stdv=False,
#                         seed = 0, early_stopping_rounds=5,show_progress = True)
# print(cvresult.tail())

### model stacking for training data
--------------------------------
- sample 10% data, use the rest as prediction
- in each fold, 40 bagging

In [15]:
# train_pivot_xgb_time1_sample = train_pivot_xgb_time1[predictors_target_10].sample(100000)
# print train_pivot_xgb_time1_sample.index
# len(train_pivot_xgb_time1_sample.loc[train_pivot_xgb_time1_sample.index.values].index.drop_duplicates())

Int64Index([17287656,  4346132,  8566861, 19992474, 15626037, 14513582,
             4768231,  4296957,  9765272,  9454972,
            ...
            14300533,  8217200, 20266266, 12168988, 17641083, 11403739,
             6318002, 16178418, 18167686, 12633318],
           dtype='int64', length=100000)


100000

In [9]:
num_round = 400
i = 0
d_train_pivot_xgb_time1 = xgb.DMatrix(train_pivot_xgb_time1[predictors_10], missing=np.nan)
gc.collect()
submission_10 = pd.DataFrame()
for i in range(40):
    train_pivot_xgb_time1_sample = train_pivot_xgb_time1[predictors_target_10].sample(2000000)

    train_feature_10 = train_pivot_xgb_time1_sample.drop(['target'],axis = 1)
    train_label_10 = train_pivot_xgb_time1_sample['target']

    dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)
    gc.collect()
    
    bst = xgb.train(param_10, dtrain_10, num_round)
    print str(i) + 'training finished!'
    gc.collect()
    
    submission_10['predict_' + str(i)] = bst.predict(d_train_pivot_xgb_time1)
    submission_10['predict_' + str(i)].loc[train_pivot_xgb_time1_sample.index.values] = np.nan
    print str(i) + 'predicting finished!'
    gc.collect()

print 'finished'

0training finished!
0predicting finished!
1training finished!
1predicting finished!
2training finished!
2predicting finished!
3training finished!
3predicting finished!
4training finished!
4predicting finished!
5training finished!
5predicting finished!
6training finished!
6predicting finished!
7training finished!
7predicting finished!
8training finished!
8predicting finished!
9training finished!
9predicting finished!
10training finished!
10predicting finished!
11training finished!
11predicting finished!
12training finished!
12predicting finished!
13training finished!
13predicting finished!
14training finished!
14predicting finished!
15training finished!
15predicting finished!
16training finished!
16predicting finished!
17training finished!
17predicting finished!
18training finished!
18predicting finished!
19training finished!
19predicting finished!
20training finished!
20predicting finished!
21training finished!
21predicting finished!
22training finished!
22predicting finished!
23traini

In [10]:
submission_10.head()

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,...,predict_30,predict_31,predict_32,predict_33,predict_34,predict_35,predict_36,predict_37,predict_38,predict_39
0,2.560112,2.319709,2.596576,3.334877,2.947561,2.540267,2.928454,2.941043,2.376304,2.995800,...,NaN,NaN,2.968065,2.672205,2.867753,2.462488,2.426948,2.630500,2.475198,2.569228
1,2.849533,2.726606,2.858416,2.941954,2.899292,2.894359,2.754009,NaN,NaN,2.725377,...,2.864190,2.890460,2.813638,2.788484,2.835742,2.866130,2.839168,2.906143,2.770211,2.806147
2,1.957559,2.057925,1.908801,2.022024,2.034622,1.946343,2.073445,1.921750,1.859954,NaN,...,2.056635,2.060785,2.062870,2.231127,1.843033,2.060290,1.876254,1.901076,1.958171,1.893032
3,3.498075,3.605519,NaN,3.481079,3.537821,3.634062,3.516993,3.518384,3.555868,3.529919,...,3.496224,3.376245,3.590961,NaN,NaN,3.428569,3.535894,3.513547,3.513322,3.568703
4,NaN,4.240009,4.264554,4.289259,4.356047,4.357327,4.233587,4.251253,4.242721,4.318971,...,4.333437,4.368112,4.320409,4.339425,4.292994,4.228761,4.276141,NaN,4.339606,4.211953


In [12]:
submission_10['predict'] = submission_10[['predict_' + str(i) for i in range(40)]].mean(axis=1)

In [14]:
submission_10.head()

,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9,...,predict_31,predict_32,predict_33,predict_34,predict_35,predict_36,predict_37,predict_38,predict_39,predict
0,2.560112,2.319709,2.596576,3.334877,2.947561,2.540267,2.928454,2.941043,2.376304,2.995800,...,NaN,2.968065,2.672205,2.867753,2.462488,2.426948,2.630500,2.475198,2.569228,2.767780
1,2.849533,2.726606,2.858416,2.941954,2.899292,2.894359,2.754009,NaN,NaN,2.725377,...,2.890460,2.813638,2.788484,2.835742,2.866130,2.839168,2.906143,2.770211,2.806147,2.835551
2,1.957559,2.057925,1.908801,2.022024,2.034622,1.946343,2.073445,1.921750,1.859954,NaN,...,2.060785,2.062870,2.231127,1.843033,2.060290,1.876254,1.901076,1.958171,1.893032,1.999626
3,3.498075,3.605519,NaN,3.481079,3.537821,3.634062,3.516993,3.518384,3.555868,3.529919,...,3.376245,3.590961,NaN,NaN,3.428569,3.535894,3.513547,3.513322,3.568703,3.505517
4,NaN,4.240009,4.264554,4.289259,4.356047,4.357327,4.233587,4.251253,4.242721,4.318971,...,4.368112,4.320409,4.339425,4.292994,4.228761,4.276141,NaN,4.339606,4.211953,4.278482


In [16]:
submission_10_final = submission_10['predict']

In [17]:
submission_10_final.head()

0    2.767780
1    2.835551
2    1.999626
3    3.505517
4    4.278482
Name: predict, dtype: float32

In [18]:
submission_10_final.to_csv('stack_train_xgb_10.csv')

### over model stacking
-----------

In [21]:
num_round = 392

dtest_10 = xgb.DMatrix(test_dataset_10_normalize[predictors_10], missing=np.nan)
submission_10 = train_pivot_56789_to_10[['id']].copy()
i = 0

for i in range(20):
    train_pivot_xgb_time1_sample = train_dataset_10_normalize[predictors_target_10].sample(2000000)

    train_feature_10 = train_pivot_xgb_time1_sample.drop(['target'],axis = 1)
    train_label_10 = train_pivot_xgb_time1_sample[['target']]

    dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)
    
    bst = xgb.train(param_10, dtrain_10, num_round)
    print str(i) + 'training finished!'
    submission_10['predict_' + str(i)] = bst.predict(dtest_10)
    print str(i) + 'predicting finished!'


print 'finished'

0training finished!
0predicting finished!
1training finished!
1predicting finished!
2training finished!
2predicting finished!
3training finished!
3predicting finished!
4training finished!
4predicting finished!
5training finished!
5predicting finished!
6training finished!
6predicting finished!
7training finished!
7predicting finished!
8training finished!
8predicting finished!
9training finished!
9predicting finished!
10training finished!
10predicting finished!
11training finished!
11predicting finished!
12training finished!
12predicting finished!
13training finished!
13predicting finished!
14training finished!
14predicting finished!
15training finished!
15predicting finished!
16training finished!
16predicting finished!
17training finished!
17predicting finished!
18training finished!
18predicting finished!
19training finished!
19predicting finished!
finished


In [22]:
submission_10.to_csv('submission_10_new.csv')

In [26]:
# make prediction
xgb.plot_importance(bst)

### combine week10 and week11
------------------

In [12]:
submission_10 = pd.read_csv('submission_10.csv',index_col = 0)

In [22]:
submission_10.shape

(3538385, 21)

In [23]:
submission_10.columns.values

array(['id', 'predict_0', 'predict_1', 'predict_2', 'predict_3',
       'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8',
       'predict_9', 'predict_10', 'predict_11', 'predict_12', 'predict_13',
       'predict_14', 'predict_15', 'predict_16', 'predict_17',
       'predict_18', 'predict_19'], dtype=object)

In [26]:
submission_11.columns.values

array(['id', 'predict_0', 'predict_1', 'predict_2', 'predict_3',
       'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8',
       'predict_9', 'predict_10', 'predict_11', 'predict_12', 'predict_13',
       'predict_14', 'predict_15', 'predict_16', 'predict_17',
       'predict_18', 'predict_19'], dtype=object)

In [27]:
submission = pd.concat([submission_10,submission_11],axis = 0)

In [28]:
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_10,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.337900,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.617610,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.030976,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.217747,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.626101,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554


In [29]:
submission['predict'] = submission[['predict_' + str(i) for i in range(20)]].mean(axis=1)

In [30]:
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19,predict
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057,2.122850
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515,3.627489
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709,2.989878
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020,4.173270
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554,3.633988


In [31]:
submission.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)

In [32]:
submission['Demanda_uni_equil'] = submission['Demanda_uni_equil'].apply(np.expm1)
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19,Demanda_uni_equil
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057,7.354917
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515,36.618239
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709,18.883251
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020,63.927416
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554,36.863517


In [33]:
submission_final = submission[['id','Demanda_uni_equil']].copy()

In [34]:
submission_final['Demanda_uni_equil'] = submission_final['Demanda_uni_equil'].round(1)

In [35]:
submission_final.head()

Semana,id,Demanda_uni_equil
0,1569352,7.4
1,6667200,36.6
2,1592616,18.9
3,3909690,63.9
4,3659672,36.9


In [36]:
submission_final.to_csv('submission_xgb_2.csv',index = False)

### read_test
----------------

In [84]:
test_id = pd.read_csv('origin/test.csv',usecols = ['id'])

In [88]:
test_id['id'].dtype

dtype('int64')

In [91]:
len(np.intersect1d(list(submission['id']), list(test_id['id'])))

5634038

# merge the other result
------------------

In [22]:
result_no_cli_pro_in_common = pd.read_csv('origin/results1.csv')
result_no_cli_pro_in_common.head()

,id,Demanda_uni_equil
0,1288534,13.5
1,1127055,359.1
2,5423268,2.9
3,749471,312.4
4,6193378,4.4


In [23]:
result_no_cli_pro_in_common = result_no_cli_pro_in_common[-result_no_cli_pro_in_common['id'].isin(np.array(submission['id']))]
result_no_cli_pro_in_common.head()

,id,Demanda_uni_equil
0,1288534,13.5
6,239863,4.4
7,2540577,3.6
8,3851172,3.4
11,6418979,3.3


In [24]:
result_no_cli_pro_in_common.shape

(1365213, 2)

In [30]:
submission_final = pd.concat([submission[['id','Demanda_uni_equil']],result_no_cli_pro_in_common],axis = 0)

In [31]:
submission_final['Demanda_uni_equil'] = submission_final['Demanda_uni_equil'].round(1)

In [32]:
submission_final.shape

(6999251, 2)

In [33]:
submission_final.to_csv('submission_xgb.csv',index = False)